<a href="https://colab.research.google.com/github/Maman08/dailyCode/blob/main/textLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import requests


In [ ]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text.lower()
chars =  sorted(set(text))

total_chars= len(chars)

char_to_idx = {char : idx for idx,char in enumerate(chars)}
idx_to_char = {idx : char for idx,char in enumerate(chars)}

window = 100

# x,y=[],[]
# for i in range (len(data_scaled) - window_len):
#   x.append(data_scaled[i : i+window_len])
#   y.append(data_scaled[i+window_len])

sequence , next_chars =[],[]
for i in range(len(text) - window):
  sequence.append([char_to_idx[c] for c in text[i:i+window]])
  next_chars.append(char_to_idx[text[i+window]])

X = np.array(sequence)
y = np.array(next_chars)
X = X / total_chars
X = X.reshape(X.shape[0],window,1)
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(window, 1), return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(total_chars, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.fit(X, y, batch_size=64, epochs=50)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 189s 11ms/step - loss: 2.6094
Epoch 2/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 201s 11ms/step - loss: 2.5001
Epoch 3/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 202s 11ms/step - loss: 2.1138
Epoch 4/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 188s 11ms/step - loss: 1.9585
Epoch 5/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 201s 11ms/step - loss: 1.8697
Epoch 6/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 202s 11ms/step - loss: 1.8089
Epoch 7/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 202s 11ms/step - loss: 1.7618
Epoch 8/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 201s 11ms/step - loss: 1.7372
Epoch 9/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 202s 11ms/step - loss: 1.6948
Epoch 10/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 201s 11ms/step - loss: 1.6730
Epoch 11/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 203s 11ms/step - loss: 1.6534
Epoch 12/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 188s 11ms/step - loss: 1.6341
Epoch 13/50
17427/17427 ━━━━━━━━━━━━━━━━━━━━ 188s 11ms/step - loss: 1.6193
Epoch 14/50
17427/17427 ━━━━━━━━━━

In [4]:
def generate_text(seed_text, length=300):
    generated = seed_text.lower()

    if len(generated) < window:
        generated = " " * (window - len(generated)) + generated

    for _ in range(length):
        input_seq = [char_to_idx.get(c, 0) for c in generated[-window:]]
        input_seq = np.array(input_seq) / total_chars
        input_seq = input_seq.reshape((1, window, 1))

        prediction = model.predict(input_seq, verbose=0)
        predicted_idx = np.argmax(prediction)
        generated += idx_to_char[predicted_idx]
    return generated


print(generate_text("shall i compare thee to a summer's day? "))

model.summary()

                                                            shall i compare thee to a summer's day? and the shame is thee to the shring the shame to the shame to the shame, and the shame to the counters of the shame and the shame with the shings
of the world be the shame with the shame to the shring
and the counters of the counters of the shings
of the counters of the counters of the shings
of the


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 39)             │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 609,527 (2.33 MB)

 Trainable params: 203,175 (793.65 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 406,352 (1.55 MB)